# **Car Brand Detection and Classification with YOLOv4**

In [1]:
# Uninstall and install compatible versions of NumPy, OpenCV, Gradio, and Matplotlib
!pip uninstall numpy opencv-python opencv-contrib-python gradio matplotlib -y
!pip install numpy==1.26.4 opencv-contrib-python==4.7.0.72 gradio matplotlib


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86
Found existing installation: opencv-contrib-python 4.11.0.86
Uninstalling opencv-contrib-python-4.11.0.86:
  Successfully uninstalled opencv-contrib-python-4.11.0.86
Found existing installation: matplotlib 3.10.0
Uninstalling matplotlib-3.10.0:
  Successfully uninstalled matplotlib-3.10.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 527.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 107.8 MB

In [ ]:
# Restart the runtime to ensure the new versions are loaded
import os
os.kill(os.getpid(), 9)  # This restarts the Colab runtime

In [1]:

# Import libraries
import cv2
import numpy as np
import time
import os
from IPython.display import display
from google.colab import drive
import gradio as gr
from collections import Counter
import matplotlib.pyplot as plt

# Verify versions
print("NumPy version:", np.__version__)
print("OpenCV version:", cv2.__version__)

# Mount Google Drive (only if not already mounted)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Define file paths for YOLOv4 files
base_path = '/content/drive/MyDrive/Car_Brand_Detection'
use_custom_model = False  # Set to True to use yolo-obj.cfg and yolo-obj_final.weights after fixing
if use_custom_model:
    config_path = f'{base_path}/cfg/yolo-obj.cfg'
    weights_path = f'{base_path}/weights/yolo-obj_final.weights'
    required_files = {
        'yolo-obj.cfg': config_path,
        'yolo-obj_final.weights': weights_path,
        'obj.names': f'{base_path}/data/obj.names'
    }
else:
    config_path = f'{base_path}/cfg/yolov4.cfg'
    weights_path = f'{base_path}/weights/yolov4.weights'
    required_files = {
        'yolov4.cfg': config_path,
        'yolov4.weights': weights_path,
        'obj.names': f'{base_path}/data/obj.names'
    }
labels_path = required_files['obj.names']

# Create obj.names with 'car' if not present (for pre-trained model)
if not os.path.exists(labels_path):
    print("[INFO] Creating default obj.names with 'car'...")
    os.makedirs(os.path.dirname(labels_path), exist_ok=True)
    with open(labels_path, 'w') as f:
        f.write('car\n')

# Verify YOLOv4 files exist
missing_files = [name for name, path in required_files.items() if not os.path.exists(path)]
if missing_files:
    raise FileNotFoundError(f"Missing files: {missing_files}. Please ensure all files are in the correct Google Drive directories.")

# Create output directories in Google Drive
os.makedirs(f'{base_path}/output_video', exist_ok=True)
os.makedirs(f'{base_path}/output_charts', exist_ok=True)
os.makedirs(f'{base_path}/debug_images', exist_ok=True)

# Constants
CONFIDENCE = 0.7
SCORE_THRESHOLD = 0.7
IOU_THRESHOLD = 0.7

# Load class labels
with open(labels_path, 'r') as f:
    LABELS = f.read().strip().split("\n")
print("[INFO] Loaded labels:", LABELS)

# Validate .cfg file classes (for custom model)
if use_custom_model:
    with open(config_path, 'r') as f:
        cfg_lines = f.readlines()
    cfg_classes = None
    for line in cfg_lines:
        if line.strip().startswith('classes='):
            cfg_classes = int(line.strip().split('=')[1])
            break
    if cfg_classes != len(LABELS):
        raise ValueError(f"Mismatch between .cfg classes ({cfg_classes}) and obj.names classes ({len(LABELS)}). Update yolo-obj.cfg or obj.names.")

# Generate colors for each class
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

# Load YOLO network
print("[INFO] Loading YOLO from disk...")
try:
    net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
    ln = net.getLayerNames()
    ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
except cv2.error as e:
    raise Exception(f"Failed to load YOLOv4 model: {str(e)}. Ensure the .cfg and .weights files are compatible with OpenCV 4.7.0. Try using standard yolov4.cfg and yolov4.weights from https://github.com/AlexeyAB/darknet.")

# Function to generate pie chart
def generate_pie_chart(brand_counts, output_path):
    if not brand_counts:
        return None, "No vehicles detected for pie chart."
    labels = list(brand_counts.keys())
    counts = list(brand_counts.values())
    total = sum(counts)
    percentages = [(count / total) * 100 for count in counts]

    plt.figure(figsize=(8, 8))
    plt.pie(counts, labels=[f"{label} ({percent:.1f}%)" for label, percent in zip(labels, percentages)],
            colors=[plt.cm.tab20(i / len(labels)) for i in range(len(labels))], startangle=140)
    plt.title("Vehicle Brand Distribution")
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()
    return output_path, "Pie chart generated successfully."

# Function to process image
def process_image(image):
    if image is None:
        return None, None, "Error: No image provided."
    # Save uploaded image temporarily
    temp_image_path = "/content/temp_image.jpg"
    cv2.imwrite(temp_image_path, image)
    output_path = f"{base_path}/output_image_yolov4.jpg"
    pie_chart_path = f"{base_path}/output_charts/image_pie_chart.png"
    debug_path = f"{base_path}/debug_images/debug_image_{int(time.time())}.jpg"

    h, w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layer_outputs = net.forward(ln)
    end = time.time()
    boxes, confidences, class_ids = [], [], []
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > CONFIDENCE:
                box = detection[:4] * np.array([w, h, w, h])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                print(f"[DEBUG] Detected: Class ID={class_id}, Label={LABELS[class_id]}, Confidence={confidence:.2f}")
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, SCORE_THRESHOLD, IOU_THRESHOLD)
    font_scale = 1
    thickness = 1
    font_color = (255, 255, 255)
    vehicle_count = len(idxs) if len(idxs) > 0 else 0
    brand_counts = Counter(LABELS[class_ids[i]] for i in idxs.flatten()) if len(idxs) > 0 else Counter()
    brand_summary = ", ".join(f"{brand}: {count}" for brand, count in brand_counts.items()) if brand_counts else "None"
    if len(idxs) > 0:
        for i in idxs.flatten():
            x, y = boxes[i][0], boxes[i][1]
            w, h = boxes[i][2], boxes[i][3]
            color = [int(c) for c in COLORS[class_ids[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)
            text = f"{LABELS[class_ids[i]]}: {confidences[i]:.2f}"
            (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
            text_offset_x, text_offset_y = x, y - 5
            box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
            overlay = image.copy()
            cv2.rectangle(overlay, box_coords[0], box_coords[1], color, cv2.FILLED)
            image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_DUPLEX, font_scale, font_color, thickness)
    cv2.imwrite(output_path, image)
    cv2.imwrite(debug_path, image)
    print(f"[DEBUG] Saved debug image: {debug_path}")

    # Generate pie chart
    pie_chart_output, pie_chart_status = generate_pie_chart(brand_counts, pie_chart_path)

    status = (f"Image processed successfully in {end - start:.2f}s.\n"
              f"Vehicles detected: {vehicle_count}\n"
              f"Brands detected: {brand_summary}\n"
              f"Pie chart status: {pie_chart_status}")
    return output_path, pie_chart_output, status

# Function to process video
def process_video(video_path):
    if video_path is None:
        return None, None, "Error: No video provided."
    # Define output paths
    output_path = f"{base_path}/output_video/processed_video.avi"
    pie_chart_path = f"{base_path}/output_charts/video_pie_chart.png"

    vs = cv2.VideoCapture(video_path)
    if not vs.isOpened():
        return None, None, f"Error: Could not open video at {video_path}"
    writer = None
    (W, H) = (None, None)
    brand_counts = Counter()
    start = time.time()
    frame_count = 0
    while True:
        grabbed, frame = vs.read()
        if not grabbed:
            break
        frame_count += 1
        if W is None or H is None:
            (H, W) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        layer_outputs = net.forward(ln)
        boxes, confidences, class_ids = [], [], []
        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > CONFIDENCE:
                    box = detection[:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
                    print(f"[DEBUG] Frame {frame_count}: Detected: Class ID={class_id}, Label={LABELS[class_id]}, Confidence={confidence:.2f}")
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, SCORE_THRESHOLD, IOU_THRESHOLD)
        if len(idxs) > 0:
            for i in idxs.flatten():
                brand_counts[LABELS[class_ids[i]]] += 1
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                color = [int(c) for c in COLORS[class_ids[i]]]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                text = f"{LABELS[class_ids[i]]}:{confidences[i]:.4f}"
                cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(output_path, fourcc, 30, (frame.shape[1], frame.shape[0]), True)
        writer.write(frame)
    end = time.time()
    print("[INFO] Cleaning up...")
    writer.release()
    vs.release()

    # Generate pie chart
    pie_chart_output, pie_chart_status = generate_pie_chart(brand_counts, pie_chart_path)

    brand_summary = ", ".join(f"{brand}: {count}" for brand, count in brand_counts.items()) if brand_counts else "None"
    status = (f"Video processed successfully in {end - start:.2f}s.\n"
              f"Brands detected: {brand_summary}\n"
              f"Pie chart status: {pie_chart_status}")
    return output_path, pie_chart_output, status

# Gradio interface function
def car_brand_detection(image, video):
    image_output, image_pie_chart, image_status = process_image(image) if image is not None else (None, None, "No image uploaded.")
    video_output, video_pie_chart, video_status = process_video(video) if video is not None else (None, None, "No video uploaded.")
    return image_output, image_pie_chart, video_output, video_pie_chart, f"{image_status}\n\n{video_status}"

# Create Gradio interface
iface = gr.Interface(
    fn=car_brand_detection,
    inputs=[
        gr.Image(type="numpy", label="Upload Image"),
        gr.Video(label="Upload Video")
    ],
    outputs=[
        gr.Image(label="Processed Image"),
        gr.Image(label="Image Vehicle Distribution Pie Chart"),
        gr.Video(label="Processed Video"),
        gr.Image(label="Video Vehicle Distribution Pie Chart"),
        gr.Textbox(label="Processing Status")
    ],
    title="Car Brand Detection and Classification with YOLOv4",
    description="Upload an image and/or video to detect car brands using YOLOv4. Outputs include processed media, detection statistics, and a pie chart showing vehicle brand distribution. Debug images are saved to Google Drive for inspection."
)

# Launch Gradio interface
iface.launch()

NumPy version: 1.26.4
OpenCV version: 4.7.0
[INFO] Loaded labels: ['Audi', 'BMW', 'Bentley', 'Chrysler', 'Ford', 'Honda', 'Hyundai', 'Mercedes-Benz', 'Nissan', 'Toyota']
[INFO] Loading YOLO from disk...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d0af1ffd2c24a5bbb4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




---



---

